In [1]:
import joblib
import numpy as np
import csv
import os
import json
import librosa
import IPython.display as ipd
import numpy as np
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
from numpy.lib import stride_tricks
from sklearn.preprocessing import normalize
import pandas as pd
import matplotlib
import cv2

""" short time fourier transform of audio signal """
def stft(sig, frameSize, overlapFac=0.9, window=np.hanning): # was 0.5
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))

    # zeros at beginning (thus center of 1st window should be for sample nr. 0)   
    samples = np.append(np.zeros(int(np.floor(frameSize/2.0))), sig)    
    # cols for windowing
    cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))

    frames = stride_tricks.as_strided(samples, shape=(int(cols), frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win

    return np.fft.rfft(frames)    

""" scale frequency axis logarithmically """    
def logscale_spec(spec, sr=22050, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))

    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):        
        if i == len(scale)-1:
            newspec[:,i] = np.sum(spec[:,int(scale[i]):], axis=1)
        else:        
            newspec[:,i] = np.sum(spec[:,int(scale[i]):int(scale[i+1])], axis=1)

    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[int(scale[i]):])]
        else:
            freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

    return newspec, freqs

""" plot spectrogram"""
def plotstft(binsize=2**10, plotpath=None, colormap="jet", srr = 22050, sam = None):
    samplerate, samples = srr, sam

    s = stft(samples, binsize)

    sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)

    ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel

    timebins, freqbins = np.shape(ims)
    
    plt.figure(figsize=(15, 7.5))
    
    fig = plt.imshow(np.transpose(ims), origin="lower", aspect="auto", cmap=colormap, interpolation="none")
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    
    plt.savefig(plotpath, bbox_inches='tight', pad_inches = 0)
    plt.close()

    #plt.show()

    #plt.clf()

    return ims

In [2]:
cutoff_sp_name = joblib.load('cutoff_sp_name_14.pkl')
(cutoff_sp_name)

[[1053, 'CAQU'],
 [573, 'CASJ'],
 [493, 'CBCH'],
 [1319, 'DEJU'],
 [980, 'OCWA'],
 [457, 'SPTO'],
 [623, 'STJA'],
 [448, 'WAVI'],
 [441, 'WITU'],
 [435, 'WIWA'],
 [590, 'WREN'],
 [492, 'WCSP'],
 [478, 'MODO'],
 [2969, 'PSFL']]

In [3]:
def custom_noise_pad_rois(sr = 22050, dur = 5, roi_npy_path = None, lab_loc = None,no_species = None):
    roi_npy = joblib.load(roi_npy_path)
    print("Shape is: ",np.shape(roi_npy))
    roi_lst = []
    lab_lst = []
    for i in range(np.shape(roi_npy)[0]):
        
        roi_sample = roi_npy[i][0]
        play = sr*dur - (roi_sample.shape[0])
        
        st_roi = np.random.randint(play)
        sig_a = 0.001 * (np.random.randint(low = 3, high = 6))
        sig_b = 0.001 * (np.random.randint(low = 3, high = 6))
        a = np.random.normal(0,sig_a,st_roi)
        b = np.random.normal(0,sig_b, play-st_roi)
        
        padded_roi = np.concatenate((a,roi_sample,b), axis=0)

        lab = (np.zeros(no_species)) 
        lab[lab_loc] = 1
        
        x1 = (len(a)/(sr*dur)) # removing  *837
        x2 = ((len(roi_sample) + len(a))/(sr*dur))
        
        y1 = abs((((roi_npy[i][4]/11025)-1))) # removing  *407
        y2 = abs((((roi_npy[i][5]/11025)-1)))
        
        # lab_lst.append(lab)
        lab_lst.append([x1, x2, y1, y2]) # appending x1,x2,x3,x4
        roi_lst.append(padded_roi)
    
    return roi_lst, lab_lst

In [4]:
import pandas as pd
train_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])

# use padding function  in this loop
sr = 22050
dur = 5 #sec
norm = 'l2'

for i in range(len(cutoff_sp_name)):

    print('>>>',cutoff_sp_name[i][1],'<<<')
    roi_npy_path = os.path.join('/media/first/pickled_s2l_2020/2_ext_april17/','ext_' + cutoff_sp_name[i][1] + '_3_Mar2020.pkl')

    roi_lst_, lab_lst_ = custom_noise_pad_rois(sr = sr, dur = dur, roi_npy_path = roi_npy_path, lab_loc=i, no_species = len(cutoff_sp_name))
    print("DONE..padding!")
    print(np.shape(roi_lst_))
    print(np.shape(lab_lst_))
    
    for no in range(np.shape(roi_lst_)[0])[20:]: # doing this for one bird-species at a time
        
        if(no%100 == 0):
            print(no,cutoff_sp_name[i][1],"Loaded to img_dir")
        
        img_dir = '/media/first/pickled_s2l_2020/2_ext_april17/imgdir/' # path to save images

        img_pth = os.path.join(img_dir,str(no) + '_' + cutoff_sp_name[i][1]) + '.png'

        pltstft = plotstft(plotpath = img_pth, colormap="Greys", srr = 22050, sam = roi_lst_[no]) # save the plot in .png format

        x1,x2,y1,y2 = lab_lst_[no]
        img_name = str(no) + '_' + cutoff_sp_name[i][1] + '.png'
        
        train_df = train_df.append({'FileName': img_name, 
                                                'XMin': x1, 
                                                'XMax': x2, 
                                                'YMin': y2, 
                                                'YMax': y1, 
                                                'ClassName': 'Bird'}, 
                                               ignore_index=True)

    
train_df.to_csv('train.csv')    

    

>>> CAQU <<<
Shape is:  (1053, 6)
DONE..padding!
(1053, 110250)
(1053, 4)
100 CAQU Loaded to img_dir
200 CAQU Loaded to img_dir
300 CAQU Loaded to img_dir
400 CAQU Loaded to img_dir
500 CAQU Loaded to img_dir
600 CAQU Loaded to img_dir
700 CAQU Loaded to img_dir
800 CAQU Loaded to img_dir
900 CAQU Loaded to img_dir
1000 CAQU Loaded to img_dir
>>> CASJ <<<
Shape is:  (573, 6)
DONE..padding!
(573, 110250)
(573, 4)
100 CASJ Loaded to img_dir
200 CASJ Loaded to img_dir
300 CASJ Loaded to img_dir
400 CASJ Loaded to img_dir
500 CASJ Loaded to img_dir
>>> CBCH <<<
Shape is:  (493, 6)
DONE..padding!
(493, 110250)
(493, 4)
100 CBCH Loaded to img_dir
200 CBCH Loaded to img_dir
300 CBCH Loaded to img_dir
400 CBCH Loaded to img_dir
>>> DEJU <<<
Shape is:  (1319, 6)
DONE..padding!
(1319, 110250)
(1319, 4)
100 DEJU Loaded to img_dir
200 DEJU Loaded to img_dir
300 DEJU Loaded to img_dir
400 DEJU Loaded to img_dir
500 DEJU Loaded to img_dir
600 DEJU Loaded to img_dir
700 DEJU Loaded to img_dir
800 DEJ

In [4]:

val_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])

# use padding function  in this loop
sr = 22050
dur = 5 #sec
norm = 'l2'

for i in range(len(cutoff_sp_name)):

    print('>>>',cutoff_sp_name[i][1],'<<<')
    roi_npy_path = os.path.join('/media/first/pickled_s2l_2020/2_ext_april17/','ext_' + cutoff_sp_name[i][1] + '_3_Mar2020.pkl')

    roi_lst_, lab_lst_ = custom_noise_pad_rois(sr = sr, dur = dur, roi_npy_path = roi_npy_path, lab_loc=i, no_species = len(cutoff_sp_name))
    print("DONE..padding!")
    print(np.shape(roi_lst_))
    print(np.shape(lab_lst_))
    
    for no in range(np.shape(roi_lst_)[0])[:20]: # doing this for one bird-species at a time
        
        if(no%10 == 0):
            print(no,cutoff_sp_name[i][1],"Loaded to img_dir")
        
        img_dir = '/media/first/pickled_s2l_2020/2_ext_april17/imgdir/test/' # path to save images
        
        img_pth = os.path.join(img_dir,str(no) + '_' + cutoff_sp_name[i][1]) + '.png'

        pltstft = plotstft(plotpath = img_pth, colormap="Greys", srr = 22050, sam = roi_lst_[no]) # save the plot in .png format

        x1,x2,y1,y2 = lab_lst_[no]
        img_name = str(no) + '_' + cutoff_sp_name[i][1] + '.png'
        
        val_df = val_df.append({'FileName': img_name, 
                                                'XMin': x1, 
                                                'XMax': x2, 
                                                'YMin': y2, 
                                                'YMax': y1, 
                                                'ClassName': 'Bird'}, 
                                               ignore_index=True)

    
val_df.to_csv('test.csv')    
    
    

>>> CAQU <<<
Shape is:  (1053, 6)
DONE..padding!
(1053, 110250)
(1053, 4)
0 CAQU Loaded to img_dir
10 CAQU Loaded to img_dir
>>> CASJ <<<
Shape is:  (573, 6)
DONE..padding!
(573, 110250)
(573, 4)
0 CASJ Loaded to img_dir
10 CASJ Loaded to img_dir
>>> CBCH <<<
Shape is:  (493, 6)
DONE..padding!
(493, 110250)
(493, 4)
0 CBCH Loaded to img_dir
10 CBCH Loaded to img_dir
>>> DEJU <<<
Shape is:  (1319, 6)
DONE..padding!
(1319, 110250)
(1319, 4)
0 DEJU Loaded to img_dir
10 DEJU Loaded to img_dir
>>> OCWA <<<
Shape is:  (980, 6)
DONE..padding!
(980, 110250)
(980, 4)
0 OCWA Loaded to img_dir
10 OCWA Loaded to img_dir
>>> SPTO <<<
Shape is:  (457, 6)
DONE..padding!
(457, 110250)
(457, 4)
0 SPTO Loaded to img_dir
10 SPTO Loaded to img_dir
>>> STJA <<<
Shape is:  (623, 6)
DONE..padding!
(623, 110250)
(623, 4)
0 STJA Loaded to img_dir
10 STJA Loaded to img_dir
>>> WAVI <<<
Shape is:  (448, 6)
DONE..padding!
(448, 110250)
(448, 4)
0 WAVI Loaded to img_dir
10 WAVI Loaded to img_dir
>>> WITU <<<
Shape

In [ ]:
val_df.head()


In [ ]:
no = 10
img_dir = '/media/first/pickled_s2l_2020/2_ext_april17/imgdir/'

cutoff_sp_name[i][1]

In [ ]:
os.path.join(img_dir,str(no) + '_' + cutoff_sp_name[i][1]) + '.png'

In [7]:
train_df = pd.read_csv('train.csv')
img_dir = '/media/first/pickled_s2l_2020/2_ext_april17/imgdir/'

# for training
with open("annotation.txt", "w+") as f:
    for idx, row in train_df.iterrows():
        img = cv2.imread(img_dir + row['FileName'])
        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)

        google_colab_file_path = '/media/first/pickled_s2l_2020/2_ext_april17/imgdir/'
        fileName = os.path.join(google_colab_file_path, row['FileName'])
        className = row['ClassName']
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')

In [5]:
import sys

test_df = pd.read_csv('test.csv')
img_dir = '/media/first/pickled_s2l_2020/2_ext_april17/imgdir/test/'

# for test
with open("test_annotation.txt", "w+") as f:
    for idx, row in test_df.iterrows():
        sys.stdout.write(str(idx) + '\r')
        sys.stdout.flush()
        img = cv2.imread(img_dir + row['FileName'])
        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)

        google_colab_file_path = '/media/first/pickled_s2l_2020/2_ext_april17/imgdir/test/'
        fileName = os.path.join(google_colab_file_path, row['FileName'])
        className = row['ClassName']
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')